In [2]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp


In [3]:
# list all files in the current directory
print(os.listdir())

['requirements.txt', 'tests', 'gov-rfp', '.gitignore', '.env', '.gitattributes', 'venv', '.git']


In [5]:
from govrfp.rfpparser.RFPParser import RFPParser
from llama_index.llms.openai import OpenAI

In [6]:
llm = OpenAI(model="gpt-3.5-turbo")
rfp = RFPParser(
    pdf_path="govrfp/rfpparser/RFP 2032H323 Amendment.pdf",
    verbose=True
)

In [8]:
chunks = rfp.chunks()

In [11]:
PROMPT_COMPLIANCE = (
    "The following excerpt is from the RFP document. Please find and list any compliance that you find in the following text.\n"
    "The compliance can be a technical requirement, a deadline, a budget, or any other requirement that the proposer must meet.\n"
    "--------------------\n"
    "Excerpt:\n"
    "{chunk}\n"
    "--------------------\n"
    "If multiple compliances are found, separate them by a new line. Start each list item with a hyphen.\n"
)

In [13]:
import tqdm
compliances = []
for chunk in tqdm.tqdm(chunks):
    prompt = PROMPT_COMPLIANCE.format(chunk=chunk['text'])
    response = llm.complete(prompt).text
    _temp = response.split("\n")
    compliances.extend([c for c in _temp if c])

  0%|          | 0/292 [00:00<?, ?it/s]

100%|██████████| 292/292 [06:51<00:00,  1.41s/it]


In [16]:
compliances

['- Deadline: 08/31/2023 1700 ET',
 '- Payment will be made by Facility Code 17b',
 '- Method of solicitation: RFQ',
 '- Size standard: % for SE',
 '- NAICS: Small Business',
 '- Offer due date: 08/31/2023 1700 ET',
 '- Small business requirement (NAICS)',
 '- Information call for solicitation',
 '- Offeror to complete specific blocks (12, 17, 23, 24, & 30)',
 '- OFFEROR TO COMPLETE BLOCKS 12, 17, 23, 24, & 30',
 '- SUBMIT INVOICES TO ADDRESS SHOWN IN BLOCK 18a UNLESS BLOCK BELOW ISSUE DATE',
 '- Submit invoices to address shown in block 18a',
 '- Delivery for FOB destination unless block is marked',
 '- This contract is a rated order under DPAS (15 CFR 700)',
 '- Indefinite Delivery Indefinite Quantity (IDIQ) contract for On Demand Coaching Services for the Treasury Executive Institute',
 '- Period of Performance: 9/30/2023 - 9/29/2027',
 '- One (1) 12-month base period, plus three (3) 12-month option periods',
 '- CONTRACTOR AGREES TO FURNISH AND DELIVER',
 '- CONTRACT/PURCHASE ORDER

In [17]:
# save the compliances to a file
with open("tests/mar-12/compliances.txt", "w") as f:
    f.write("\n".join(compliances))

In [18]:
PROMPT_ACTIONABLE_COMPLIANCE = (
    "For the compliances listed below, extracted from an RFP document, return the actionable ones.\n"
    "For example, if the compliance is a deadline, the actionable compliance would be the date of the deadline.\n"
    "If the compliance is a technical requirement, the actionable compliance would be the specific requirement.\n"
    "If the compliance refers to some other document, it is not actionable. return None.\n"
    "{compliances}\n"
    "---\n"
    "If multiple compliances are found, separate them by a new line. Start each list item with a hyphen.\n"
)

In [20]:
batch_size = 10
actionable_compliances = []
for i in tqdm.tqdm(range(0, len(compliances), batch_size)):
    batch = compliances[i:i+batch_size]
    prompt = PROMPT_ACTIONABLE_COMPLIANCE.format(compliances="\n".join(batch))
    response = llm.complete(prompt).text
    _temp = response.split("\n")
    actionable_compliances.extend([c for c in _temp if c])

100%|██████████| 91/91 [03:29<00:00,  2.31s/it]


In [23]:
actionable_compliances

['- Deadline: 08/31/2023 1700 ET',
 '- Payment will be made by Facility Code 17b',
 '- Method of solicitation: RFQ',
 '- Size standard: % for SE',
 '- NAICS: Small Business',
 '- Offer due date: 08/31/2023 1700 ET',
 '- Small business requirement (NAICS)',
 '- Offeror to complete specific blocks (12, 17, 23, 24, & 30)',
 '- SUBMIT INVOICES TO ADDRESS SHOWN IN BLOCK 18a UNLESS BLOCK BELOW ISSUE DATE',
 '- Submit invoices to address shown in block 18a',
 '- Delivery for FOB destination unless block is marked',
 '- This contract is a rated order under DPAS (15 CFR 700)',
 '- Indefinite Delivery Indefinite Quantity (IDIQ) contract for On Demand Coaching Services for the Treasury Executive Institute',
 '- Period of Performance: 9/30/2023 - 9/29/2027',
 '- One (1) 12-month base period, plus three (3) 12-month option periods',
 '- CONTRACTOR AGREES TO FURNISH AND DELIVER',
 '- CONTRACT/PURCHASE ORDER INCORPORATES BY REFERENCE FAR 52.212-4',
 '- The offer must be submitted on Standard Form 144

In [24]:
# save the actionable compliances to a file
with open("tests/mar-12/actionable_compliances.txt", "w") as f:
    f.write("\n".join(actionable_compliances))

In [35]:
PROMPT3 = (
    "For the given compliances, extracted from an RFP document, return a final set of compliances that the applicant will have to adhere to.\n"
    "The compliances should be actionable and should be in a format that can be used to create a checklist.\n"
    "For example, if the compliance is a deadline, the actionable compliance would be the date of the deadline.\n"
    "Examples of required compliances:\n"
    "- Deadline: 08/31/2023 1700 ET\n"
    "- Cover Letter: 2 pages maximum\n"
    "- Blank pages will contain the marking “Page Intentionally Left Blank.”\n"
    "--------------------\n"
    "Examples of non-actionable compliances:\n"
    "- Meets 2.3.2 COTS On Demand Coaching Platform requirements\n"
    "- Total Amount of Compensation\n"
    "- Full payment shall be made for 100% compliance for each weekly and monthly deliverable.\n"
    "--------------------\n"
    "Given compliances:\n"
    "{actionable_compliances}\n"
    "--------------------\n"
    "If multiple compliances are found, separate them by a new line. Start each list item with a hyphen.\n"
    "For non-actionable compliances, return None.\n"
    "Make the returned compliances human-readable and standalone.\n"
    "--------------------\n"
)

In [37]:
batch_size = 20
final_compliances = []
for i in tqdm.tqdm(range(0, len(actionable_compliances), batch_size)):
    batch = actionable_compliances[i:i+batch_size]
    prompt = PROMPT3.format(actionable_compliances="\n".join(batch))
    response = llm.complete(prompt).text
    _temp = response.split("\n")
    final_compliances.extend([c for c in _temp if c])

100%|██████████| 28/28 [02:13<00:00,  4.78s/it]


In [38]:
# save the final compliances to a file
with open("tests/mar-12/final_compliances.txt", "w") as f:
    f.write("\n".join(final_compliances))

In [39]:
final_compliances

['- Deadline: 08/31/2023 1700 ET',
 '- Offer due date: 08/31/2023 1700 ET',
 '- Submit invoices to address shown in block 18a',
 '- Delivery for FOB destination unless block is marked',
 '- Indefinite Delivery Indefinite Quantity (IDIQ) contract for On Demand Coaching Services for the Treasury Executive Institute',
 '- Period of Performance: 9/30/2023 - 9/29/2027',
 '- One (1) 12-month base period, plus three (3) 12-month option periods',
 '- The offer must be submitted on Standard Form 1449 (REV. 2/2012)',
 '- The contractor is required to sign the document and return it',
 '- The proposer must provide the mailing address of the authorized government representative.',
 '- Include the date and signature of the authorized government representative',
 '- Ensure that the quantity in column 21 has been received and inspected',
 '- Provide the e-mail of the authorized government representative',
 '- Include the total number of containers, date received, location received at, and the name of

In [50]:
PROMPT_4 = (
    "Given the following compliances, return any compliances that are required while writing an application for the RFP.\n"
    "Return None, if no such compliances are found.\n"
    "--------------------\n"
    "Example 1:\n"
    "Given compliances:\n"
    "- Deadline: 08/31/2023 1700 ET\n"
    "- Offer due date: 08/31/2023 1700 ET\n"
    "- Submit invoices to address shown in block 18a\n"
    "Response:\n"
    "- Deadline is 5:00 PM ET on August 31, 2023\n"
    "--------------------\n"
    "Example 2:\n"
    "Given compliances:\n"
    '- Any deviation or delay upon the agreed upon delivery time will be recorded and influence the contractor closing assessment made by the COR\n'
    '- Offeror must provide description of service(s) or product(s)\n'
    '- Number and Names of Sub-contractors, if any\n'
    "Response:\n"
    "- None\n"
    "--------------------\n"
    "Given compliances:\n"
    "- {final_compliances}\n"
    "--------------------\n"
    "If multiple compliances are found, separate them by a new line. Start each list item with a hyphen.\n"
    "Combine similar compliances into one.\n"
    "Response:\n"
)

In [51]:
llm = OpenAI(model="gpt-4")
batch_size = 20
final_compliances2 = []
for i in tqdm.tqdm(range(0, len(actionable_compliances), batch_size)):
    batch = actionable_compliances[i:i+batch_size]
    prompt = PROMPT_4.format(final_compliances="\n- ".join(batch))
    response = llm.complete(prompt).text
    print(response)
    _temp = response.split("\n")
    final_compliances2.extend([c for c in _temp if c])

  4%|▎         | 1/28 [00:03<01:43,  3.82s/it]

- Deadline is 5:00 PM ET on August 31, 2023
- Submit invoices to address shown in block 18a
- The offer must be submitted on Standard Form 1449 (REV. 2/2012)
- The contractor is required to sign the document and return it
- The proposer must provide the mailing address of the authorized government representative.


  7%|▋         | 2/28 [00:11<02:41,  6.21s/it]

- The proposer must include the date and signature of the authorized government representative.
- The proposer must ensure that the quantity in column 21 has been received and inspected.
- The proposer must provide the e-mail of the authorized government representative.
- The proposer must include the total number of containers, date received, location received at, and the name of the person who received the containers.
- The proposer must include the date, signature, and title of the certifying officer.
- The proposer must certify that the account is correct and proper for payment.
- Period of Performance: From September 30, 2023 to September 30, 2027.
- Licenses required: Executive Coaching/Assessments/Learning Resources in accordance with PWS.
- Deadlines for Option Periods: 
  - Option Period 1: September 30, 2024
  - Option Period 2: September 30, 2025
  - Option Period 3: September 30, 2026
- Comply with FAR 52.216-19 Order Limitations (Oct 1995) and FAR 52.217-8 Option to Extend

 11%|█         | 3/28 [00:17<02:28,  5.95s/it]

...

- The Contractor is not authorized to commence performance prior to the issuance of a signed Task Order or other written approval provided by the CO to begin work.
- This service must meet industry standards to comply with 508 regulations.
- The COR must be appointed and identified, and the performance of work under the contract is subject to their technical direction. Any oral direction must be confirmed in writing within five workdays.
- The Contractor shall provide a Project Manager for this contract who will be responsible for the overall management and coordination, act as the central point of contact with the Government, and have full authority to act for the Contractor in the performance of the required services. They must respond within four hours after notification of a problem.
- Invoices must be submitted electronically to https://www.ipp.gov and an electronic copy of the invoice must be simultaneously submitted to the COR and the Contracting Officer. Each invoice submitted 

 25%|██▌       | 7/28 [00:57<03:17,  9.43s/it]

- Payment requests must include supporting documentation and must be submitted electronically using the Invoice Processing Platform (IPP) in the proper format containing the required information for payment processing.
- Invoices must meet the minimum standards specified in FAR 32.905(b) and certain documents are required to be attached to the invoice.
- If unable to use IPP for submitting payment requests, a waiver form must be completed and submitted for review and approval by the Contracting Officer in Adobe Acrobat Portable Document Format (PDF) or Microsoft Word.
- Vendors will be notified regarding the approval or rejection of their waiver within a reasonable number of days. If a waiver is granted, a copy of the waiver must be submitted with each invoice to the payment office.
- The application must comply with the following contract terms and conditions: 52.212-4, 52.219-10, 52.227-2, 52.229-3, 52.232-23, 52.232-39, 52.232-40, 52.243-1.


 25%|██▌       | 7/28 [00:58<02:55,  8.35s/it]


KeyboardInterrupt: 

In [43]:
len(final_compliances2)

459

In [ ]:
# didn't work very well. Let's try a different approach.